<a href="https://colab.research.google.com/github/Tugume/OOP-Assignment/blob/main/Final_OOP_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

uploaded = files.upload()

Saving sales_data_sample.csv to sales_data_sample.csv


In [3]:
import pandas as pd
# Read the uploaded CSV file
df = pd.read_csv('sales_data_sample.csv', encoding='ISO-8859-1')

# Show the first 5 rows
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [4]:
# Keep only the columns we need (Date and Sales)
df = df[['ORDERDATE', 'SALES']]

# Convert the 'ORDERDATE' column to datetime format (e.g., 2/24/2003 → 2003-02-24)
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

# Group sales by date (add sales from same date)
df = df.groupby('ORDERDATE').sum()

# Sort by date (from oldest to newest)
df = df.sort_index()

# View the first few rows
print(df.head())

               SALES
ORDERDATE           
2003-01-06  12133.25
2003-01-09  11432.34
2003-01-10   6864.05
2003-01-29  54702.00
2003-01-31  44621.96
